<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#scrap-championships" data-toc-modified-id="scrap-championships-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>scrap championships</a></span></li><li><span><a href="#scrap-matches" data-toc-modified-id="scrap-matches-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>scrap matches</a></span></li><li><span><a href="#scrap-teams-and-players" data-toc-modified-id="scrap-teams-and-players-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>scrap teams and players</a></span></li><li><span><a href="#add-meteo-to-the-df_matches" data-toc-modified-id="add-meteo-to-the-df_matches-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>add meteo to the df_matches</a></span></li><li><span><a href="#create-table-match-and-matches/teams" data-toc-modified-id="create-table-match-and-matches/teams-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>create table match and matches/teams</a></span></li><li><span><a href="#add-df-to-sql-database" data-toc-modified-id="add-df-to-sql-database-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>add df to sql database</a></span><ul class="toc-item"><li><span><a href="#create-championship-table-sql" data-toc-modified-id="create-championship-table-sql-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>create championship table sql</a></span></li><li><span><a href="#create-teams-table-in-sql" data-toc-modified-id="create-teams-table-in-sql-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>create teams table in sql</a></span></li><li><span><a href="#create-participation-table" data-toc-modified-id="create-participation-table-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>create participation table</a></span></li><li><span><a href="#create-players-table" data-toc-modified-id="create-players-table-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>create players table</a></span></li><li><span><a href="#create-matches-table" data-toc-modified-id="create-matches-table-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>create matches table</a></span></li><li><span><a href="#create-matches_teams-table" data-toc-modified-id="create-matches_teams-table-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>create matches_teams table</a></span></li><li><span><a href="#create-goals-table" data-toc-modified-id="create-goals-table-6.7"><span class="toc-item-num">6.7&nbsp;&nbsp;</span>create goals table</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import matplotlib.pyplot  as plt
import seaborn as sns
import json
import requests
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq

url = "https://www.lequipe.fr/Football/"
main_url = 'https://www.lequipe.fr'

def get_page_soup(url):
    """get_page_soup(url) takes url of the web page and return it html soup.
    Parameters:
    url: <str> web page address
    Output:
    BeautifulSoup()""" 
    response = requests.get(url)
    return soup(response.text, 'html.parser')

### scrap championships

In [2]:
#get championships links

main_soup = get_page_soup(url)
#equipe_soup
#a aria-label = '/Football/' class = 'Link Menu__accordionTitle'
championships = main_soup.find("div", {"class": 'Menu__accordion js-menu-accordion is-active'}).findAll("li", {"class":"Menu__item" })
championships_dict = {'name':[], 'link':[]}
for champ in championships:
    championships_dict['name'].append(champ.a.text.split('\n')[1].strip())
    championships_dict['link'].append(champ.a['href'])

df_championships = pd.DataFrame(championships_dict)
df_championships.head()

teams_link_list = []; archives_link_list = []

for i in range(len(df_championships)):
    url_champ = df_championships.loc[i,'link']
    # insert code below here after
    
    # for single championship
    #url_champ = df_championships.loc[2,'link']
    page_championship = get_page_soup(url_champ)
    navigator_items = page_championship.find("div", {"class": "MainNav__items js-mainNavItems"}).findAll("a", {"class": "Link MainNav__item"})

    teams_link = None; archives_link = None

    for item in navigator_items:
        if "sélections" in item.text.strip().split() or "clubs" in item.text.strip().split():   
            teams_link = item["href"]
        if "archives" in item.text.strip().split():
            archives_link = item["href"]

    teams_link_list.append(teams_link)
    archives_link_list.append(archives_link)

df_championships['teams_link'] = teams_link_list
df_championships['archives_link'] = archives_link_list

df_championships.isnull().sum()

name              0
link              0
teams_link       14
archives_link    15
dtype: int64

### scrap matches 

In [3]:
# scrap archive to get matches information for each championship
archives_url = df_championships.loc[2,'archives_link']
champ_name = df_championships.loc[2,'name']
print(champ_name)

Ligue 1


In [4]:
#get links for all archive of each year of this championship
archives_page = get_page_soup(archives_url)
archives_list = archives_page.findAll("td")
for archive in archives_list:
    ligue_name = archive.h2.text
    link = archive.findAll('div', {"class":"ligneee"})[1].a['href']
    print(link)

/Football/ligue-1/saison-2019-2020/page-calendrier-resultats
/Football/ligue-1/saison-2018-2019/page-calendrier-resultats
/Football/ligue-1/saison-2017-2018/page-calendrier-resultats
/Football/ligue-1/saison-2016-2017/page-calendrier-resultats
/Football/ligue-1/saison-2015-2016/page-calendrier-resultats
/Football/ligue-1/saison-2014-2015/page-calendrier-resultats
/Football/ligue-1/saison-2013-2014/page-calendrier-resultats
/Football/ligue-1/saison-2012-2013/page-calendrier-resultats
/Football/ligue-1/saison-2011-2012/page-calendrier-resultats
/Football/ligue-1/saison-2010-2011/page-calendrier-resultats
/Football/ligue-1/saison-2009-2010/page-calendrier-resultats
/Football/ligue-1/saison-2008-2009/page-calendrier-resultats
/Football/ligue-1/saison-2007-2008/page-calendrier-resultats
/Football/ligue-1/saison-2006-2007/page-calendrier-resultats
/Football/ligue-1/saison-2005-2006/page-calendrier-resultats
/Football/ligue-1/saison-2004-2005/page-calendrier-resultats
/Football/ligue-1/saison

In [5]:
from datetime import datetime  as dt

def get_goal_list(url):
    page_soup = get_page_soup(url)
    goal_list = page_soup.findAll(
        'div', attrs={'class': 'grid__content Scoreboard__goalList'})

    goal_list_right = []
    goal_list_left = []

    if goal_list != []:
       
        for s in goal_list[0].findAll('div',{'class': 'Scoreboard__goals'}):
            name = s.find('span',{'class': 'Scoreboard__scorer'}).text
            goal_time = s.findAll('span')[2].text
            goal_type = s.findAll('span')[0].text
            goal_list_left.append((name, goal_time, goal_type))

        for s in goal_list[1].findAll('div',{'class': 'Scoreboard__goals'}):
            name = s.find('span',{'class': 'Scoreboard__scorer'}).text
            goal_time = s.findAll('span')[0].text
            goal_type = s.findAll('span')[2].text
            goal_list_right.append((name, goal_time, goal_type))

    return (goal_list_left, goal_list_right)

def fill_dict_teams_on_championship(dict_teams_on_championship, url):
    page_soup = get_page_soup(url)
    goal_list = page_soup.findAll('a', attrs={'class': 'Link TeamScore__team'})

    club_name = goal_list[0].text.strip()
    link = goal_list[0]['href']
    if club_name not in dict_teams_on_championship['team']:
        dict_teams_on_championship['team'].append(club_name)
        dict_teams_on_championship['link_on_team_page'].append(link)


link_matches = 'https://www.lequipe.fr/Football/ligue-1/saison-2020-2021/page-calendrier-resultats'
matches_page = get_page_soup(link_matches)

matches_json = []
matches = matches_page.find("select",{"class":"SelectNav__select"}).findAll('option')
for match in matches:
    matches_json.append(match['value'])
    
dict_matches = {
    'date': [],
    'place': [],
    'team_domicile': [],
    'team_exterieur': [],
    'score_domicile': [],
    'score_exterieur': [],
    'vainqueur': [],
    'link_on_match': [],
    'goals_domicile': [],
    'goals_exterieur': []
}

dict_teams_on_championship = {'team': [], 'link_on_team_page': []}

for match_json in matches_json:
    response = requests.get(match_json)
    data = json.loads(response.text)
    for day in data['items'][:-1]:        
        for game in day['items']:
            dict_matches["place"].append(game['event']['lieu']['ville'])
            dict_matches["date"].append(dt.strptime(game['date'].split('+')[0], "%Y-%m-%dT%H:%M:%S"))
            dict_matches["team_domicile"].append(game['event']['specifics']['domicile']['equipe']['nom'])
            dict_matches["team_exterieur"].append(game['event']['specifics']['exterieur']['equipe']['nom'])
            dict_matches["score_domicile"].append(game['event']['specifics']['score']['domicile'])
            dict_matches["score_exterieur"].append(game['event']['specifics']['score']['exterieur'])
            dict_matches["vainqueur"].append(game['event']['specifics']['vainqueur'])
            link_on_match = game['event']['lien_web']
            dict_matches["link_on_match"].append(link_on_match)
            
            fill_dict_teams_on_championship(dict_teams_on_championship, link_on_match)
            
            (goals_left, goals_right) = get_goal_list(link_on_match)
            dict_matches["goals_domicile"].append(goals_left)
            dict_matches["goals_exterieur"].append(goals_right)         
            
df_matches_score = pd.DataFrame(dict_matches)

In [6]:
df_teams_links = pd.DataFrame(dict_teams_on_championship)


In [7]:
df_matches_score

,date,place,team_domicile,team_exterieur,score_domicile,score_exterieur,vainqueur,link_on_match,goals_domicile,goals_exterieur
0,2020-08-21 19:00:00,Bordeaux,Bordeaux,Nantes,0,0,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...,[],[]
1,2020-08-22 17:00:00,Dijon,Dijon,Angers,0,1,exterieur,https://www.lequipe.fr/Football/match-direct/l...,[],"[(I. Traoré, 22’, )]"
2,2020-08-22 21:00:00,Lille,Lille,Rennes,1,1,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...,"[(J. Bamba, 40’, )]","[(D. Da Silva, 74’, )]"
3,2020-08-23 13:00:00,Monaco,Monaco,Reims,2,2,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...,"[(A. Disasi, 45’ +2, ), (B. Badiashile, 55’, )]","[(B. Dia, 05’, ), (E. Touré, 21’, )]"
4,2020-08-23 15:00:00,Lorient,Lorient,Strasbourg,3,1,domicile,https://www.lequipe.fr/Football/match-direct/l...,"[(Y. Wissa, 51’, ), (A. Grbic, 60’, (s.p.)), (...","[(M. Chahiri, 30’, )]"
...,...,...,...,...,...,...,...,...,...,...
375,2021-05-23 21:00:00,Rennes,Rennes,Nîmes,2,0,domicile,https://www.lequipe.fr/Football/match-direct/l...,"[(S. Guirassy, 25’, ), (B. Bourigeaud, 51’, )]",[]
376,2021-05-23 21:00:00,Nantes,Nantes,Montpellier,1,2,exterieur,https://www.lequipe.fr/Football/match-direct/l...,"[(R. Kolo Muani, 33’, )]","[(G. Laborde, 31’, ), (A. Delort, 76’, )]"
377,2021-05-23 21:00:00,Saint-Étienne,Saint-Étienne,Dijon,0,1,exterieur,https://www.lequipe.fr/Football/match-direct/l...,[],"[(A. Kamara, 39’, )]"
378,2021-05-23 21:00:00,Reims,Reims,Bordeaux,1,2,exterieur,https://www.lequipe.fr/Football/match-direct/l...,"[(E. Touré, 15’, )]","[(Y. Adli, 44’, ), (E. Kwateng, 58’, )]"


### scrap teams and players

In [8]:
# get info for each team
def change_date_format(a):
    if a is not np.nan:
        a = a.split("/")
        a[-1] = str(int(a[-1]) +
                    2000) if int(a[-1]) < 30 else str(int(a[-1]) + 1900)
        a[-2] = a[-2].zfill(2)
        a[-3] = a[-3].zfill(2)
        a.reverse()
        a = '-'.join(a)
    return (a)


dict_teams = {
    'city': [],
    'name': [],
    'coach_name': [],
    'president': [],
    'year_founded': [],
    'championship': []
}
dict_players = {
    'number': [],
    'name': [],
    'country': [],
    'birthday': [],
    'position': [],
    'club': []
}


df_players = pd.DataFrame(dict_players)
#print(df_players.head())
for i, (link, name) in enumerate(zip(df_teams_links['link_on_team_page'], df_teams_links['team'])):

    #     link = df_teams_links.loc[i, 'links']
    #     name = df_teams_links.loc[i, 'team_short_names']
    #print(name)
    # team info and player info dirty tables
    team_info = pd.read_html(link)[0]

    dict_teams['city'].append(name)
    dict_teams['name'].append(team_info.loc[0, 0].split(':')[1].strip())
    dict_teams['year_founded'].append(team_info.loc[1,
                                                    0].split(':')[1].strip())
    dict_teams['president'].append(team_info.loc[2, 0].split(':')[1].strip())
    dict_teams['coach_name'].append(team_info.loc[3, 0].split(':')[1].strip())
    dict_teams['championship'].append(champ_name)

    # players info
    if len(pd.read_html(link)[2]) > 5:
        team_players = pd.read_html(link)[2]
    else:
        team_players = pd.read_html(link)[1]

    old_column_names = team_players.columns
    team_players = team_players.rename(
        columns={
            'Unnamed: 0': 'number',
            'Nom': 'name',
            'Pays': 'country',
            old_column_names[3]: 'birthday',
            'Pos.': 'position'
        })
    team_players['club'] = name

    df_players = pd.concat([df_players, team_players], ignore_index=True)
    #print(df_players.tail(2))

df_teams = pd.DataFrame(dict_teams)

df_players['birthday'] = df_players['birthday'].apply(change_date_format)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
len(df_players)

647

In [10]:
df_teams.head()

,city,name,coach_name,president,year_founded,championship
0,Bordeaux,Football Club des Girondins de Bordeaux,Jean-Louis GASSET,Frédéric Longuépée,1881,Ligue 1
1,Dijon,Dijon Football Côte-d'Or,David LINARES,Olivier Delcourt,1998,Ligue 1
2,Lille,LOSC Lille,Christophe GALTIER,Olivier Létang,1944,Ligue 1
3,Monaco,Association sportive de Monaco FC,Niko KOVAC,Dmitri Rybolovlev,1919,Ligue 1
4,Lorient,Football Club Lorient-Bretagne Sud,Christophe PELISSIER,Loïc Féry,1926,Ligue 1


### add meteo to the df_matches

### create table match and matches/teams

In [11]:
display(df_matches_score.head())
# df_matches
df_matches = df_matches_score[['date','place']]
df_matches.is_copy = None
display(df_matches.head())

# df_matches_teams
df_matches_teams1 = df_matches_score[['date','place','team_domicile','score_domicile', 'goals_domicile']].rename(columns = {'team_domicile': 'team', 'score_domicile': 'team_goal', 'goals_domicile': 'players'})
df_matches_teams2 = df_matches_score[['date','place','team_exterieur','score_exterieur', 'goals_exterieur']].rename(columns = {'team_exterieur': 'team', 'score_exterieur': 'team_goal', 'goals_exterieur': 'players'})
df_matches_teams = pd.concat([df_matches_teams1, df_matches_teams2], ignore_index=True)
df_matches_teams = df_matches_teams.sort_values(by='date').reset_index(drop=True)
display(df_matches_teams.head(10))   

,date,place,team_domicile,team_exterieur,score_domicile,score_exterieur,vainqueur,link_on_match,goals_domicile,goals_exterieur
0,2020-08-21 19:00:00,Bordeaux,Bordeaux,Nantes,0,0,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...,[],[]
1,2020-08-22 17:00:00,Dijon,Dijon,Angers,0,1,exterieur,https://www.lequipe.fr/Football/match-direct/l...,[],"[(I. Traoré, 22’, )]"
2,2020-08-22 21:00:00,Lille,Lille,Rennes,1,1,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...,"[(J. Bamba, 40’, )]","[(D. Da Silva, 74’, )]"
3,2020-08-23 13:00:00,Monaco,Monaco,Reims,2,2,ex-aequo,https://www.lequipe.fr/Football/match-direct/l...,"[(A. Disasi, 45’ +2, ), (B. Badiashile, 55’, )]","[(B. Dia, 05’, ), (E. Touré, 21’, )]"
4,2020-08-23 15:00:00,Lorient,Lorient,Strasbourg,3,1,domicile,https://www.lequipe.fr/Football/match-direct/l...,"[(Y. Wissa, 51’, ), (A. Grbic, 60’, (s.p.)), (...","[(M. Chahiri, 30’, )]"


,date,place
0,2020-08-21 19:00:00,Bordeaux
1,2020-08-22 17:00:00,Dijon
2,2020-08-22 21:00:00,Lille
3,2020-08-23 13:00:00,Monaco
4,2020-08-23 15:00:00,Lorient


,date,place,team,team_goal,players
0,2020-08-21 19:00:00,Bordeaux,Bordeaux,0,[]
1,2020-08-21 19:00:00,Bordeaux,Nantes,0,[]
2,2020-08-22 17:00:00,Dijon,Dijon,0,[]
3,2020-08-22 17:00:00,Dijon,Angers,1,"[(I. Traoré, 22’, )]"
4,2020-08-22 21:00:00,Lille,Rennes,1,"[(D. Da Silva, 74’, )]"
5,2020-08-22 21:00:00,Lille,Lille,1,"[(J. Bamba, 40’, )]"
6,2020-08-23 13:00:00,Monaco,Monaco,2,"[(A. Disasi, 45’ +2, ), (B. Badiashile, 55’, )]"
7,2020-08-23 13:00:00,Monaco,Reims,2,"[(B. Dia, 05’, ), (E. Touré, 21’, )]"
8,2020-08-23 15:00:00,Lorient,Lorient,3,"[(Y. Wissa, 51’, ), (A. Grbic, 60’, (s.p.)), (..."
9,2020-08-23 15:00:00,Nîmes,Brest,0,[]


In [12]:
df_matches['temperature'] = [18]*len(df_matches)
df_matches['rainfall'] = [0]*len(df_matches)
df_matches.tail()
print(len(df_matches_score))

380


<ipython-input-12-8d71e27cafc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches['temperature'] = [18]*len(df_matches)
<ipython-input-12-8d71e27cafc8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matches['rainfall'] = [0]*len(df_matches)


### add df to sql database 

In [132]:
import sqlite3
class ChampionshipsDataBase:

    def __init__(self, name='championships.sqlite'):
        self.name_db = name

        if os.path.isfile(
                self.name_db):  # if sqlite file exist create a connection
            self.connection = self.__connect_db()
            self.cur = self.connection.cursor()
            print(self.connection)

        else:  # if file sqlite doesn't exist create file and then create sql

            open(self.name_db, 'x')
            self.connection = self.__connect_db()
            print(self.connection)
            self.cur = self.connection.cursor()
            self.__create_db()
            #self.fill_db()

    def get_tables_names(self):
        self.cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

        #store all names of the tables/chemas in var tables
        tables0 = self.cur.fetchall()

        tables = []
        for tab in tables0:
            tables.append(tab[0])

        return tables
    

    def get_df(self, table_name):
        return pd.read_sql_query(f"SELECT * FROM {table_name}",
                                 self.connection)

    def __connect_db(self):
        return sqlite3.connect(self.name_db)
    
    def __create_db(self):
        
        conn = self.connection
        cursor = self.cur

        #championships
        cursor.execute("""
        CREATE TABLE championships(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(255),
        country VARCHAR(255),
        start_year INTEGER,
        end_year INTEGER
        )
        ;""")

        conn.commit()
           
        #teams
        cursor.execute("""
        CREATE TABLE teams(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(255),
        city VARCHAR(255),
        coach_name VARCHAR(255),
        year_founded INTEGER
        )
        ;""")
        conn.commit()
               
        #participation
        cursor.execute("""
        CREATE TABLE participation(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        championship_id INTEGER,
        team_id INTEGER,
        FOREIGN KEY (championship_id)
        REFERENCES championships(id),
        FOREIGN KEY (team_id)
        REFERENCES teams(id)
        )
        ;""")
        conn.commit()

        
        #matches
        cursor.execute("""
        CREATE TABLE matches(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE,
        place VARCHAR(255),
        rainfall REAL,
        temperature REAL
        )
        ;""")
        conn.commit()


        #matches_teams
        cursor.execute("""
        CREATE TABLE matches_teams(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        match_id INTEGER,
        team_id INTEGER,
        home INTEGER,
        team_goal INTEGER,
        points INTEGER,
        FOREIGN KEY (match_id)
        REFERENCES matches(id),
        FOREIGN KEY (team_id)
        REFERENCES teams(id)
        )
        ;""")
        conn.commit()

        #players
        cursor.execute("""
        CREATE TABLE players(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(255),
        team_id INTEGER,
        position VARCHAR(255),
        birthday DATE,
        nationality VARCHAR(255),
        FOREIGN KEY (team_id)
        REFERENCES teams(id)
        )
        ;""")
        conn.commit()

        #goals
        cursor.execute("""
        CREATE TABLE goals(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        player_id INTEGER,
        match_id INTEGER,
        goal_time VARCHAR(255),
        goal_type VARCHAR(255),
        FOREIGN KEY (match_id)
        REFERENCES matches(id),
        FOREIGN KEY (player_id)
        REFERENCES players(id)
        )
        ;""")
        conn.commit()
        
    #1.championships
    def update_championships_table(self):
        self.get_df('championships')
        self.cur.execute("""INSERT INTO championships(name, country, start_year, end_year) 
            SELECT ?, ?, ?, ?
            WHERE NOT EXISTS(SELECT * FROM championships WHERE name = ? AND country = ? 
            AND start_year = ? AND end_year = ?);
            """,('Ligue 1', 'France', 2020, 2021, 'Ligue 1', 'France', 2020, 2021))
        
    #2. teams 
    def update_teams_table(self, df_teams):
        for i in range(len(df_teams)):
            name = df_teams.loc[i, 'name']
            city = df_teams.loc[i, 'city']
            coach_name = df_teams.loc[i, 'coach_name']
            year_founded = df_teams.loc[i, 'year_founded']

            self.cur.execute("""INSERT INTO teams(name, city, coach_name, year_founded) 
        SELECT ?, ?, ?, ?
        WHERE NOT EXISTS(SELECT * FROM teams WHERE name = ? AND city = ? AND coach_name = ? 
        AND year_founded = ?);
        """,(name, city, coach_name, year_founded, name, city, coach_name, year_founded)) 
            
    #3. participation      
    def update_participation_table(self, df_teams):
        for i in range(len(df_teams)):
            championship_id = self.cur.execute(
                "SELECT id FROM championships WHERE name = '{}';".format(
                    df_teams.loc[i, 'championship'])).fetchone()[0]
            team_id = self.cur.execute(
                "SELECT id FROM teams WHERE city = '{}';".format(
                    df_teams.loc[i, 'city'])).fetchone()[0]
            self.cur.execute("""INSERT INTO participation(championship_id, team_id) 
        SELECT '{}', '{}' 
        WHERE NOT EXISTS(SELECT * FROM participation WHERE championship_id = '{}' AND team_id = '{}');"""
                           .format(championship_id, team_id, championship_id,
                                   team_id))
    #4. players     
    def update_players_table(self, df_players):
        for i in range(len(df_players)):
            name = df_players.loc[i, 'name']
            team_id = self.cur.execute("SELECT id FROM teams WHERE city = '{}';".format(
                df_players.loc[i, 'club'])).fetchone()[0]
            position = df_players.loc[i, 'position']
            birthday = df_players.loc[i, 'birthday']
            nationality = df_players.loc[i, 'country']
            #print(name, team_id, position, birthday, nationality)

            self.cur.execute(
                """INSERT INTO players(name, team_id, position, birthday, nationality) 
                    SELECT "{}", "{}", "{}", "{}", "{}" 
                    WHERE NOT EXISTS(SELECT * FROM players WHERE name = "{}" AND team_id = "{}" 
                    AND position = "{}" AND birthday = "{}" AND nationality = "{}");"""
                .format(name, team_id, position, birthday, nationality, name, team_id,
                        position, birthday, nationality))
            
    def update_matches_table(self, df_matches):
        for i in range(len(df_matches)):
            date = df_matches.loc[i, 'date']
            place = df_matches.loc[i, 'place']
            temperature = df_matches.loc[i, 'temperature']
            rainfall = df_matches.loc[i, 'rainfall']

            self.cur.execute("""INSERT INTO matches(date, place, temperature, rainfall) 
        SELECT "{}", "{}", "{}", "{}"
        WHERE NOT EXISTS(SELECT * FROM matches WHERE date = "{}" AND place = "{}" AND temperature = "{}" 
        AND rainfall = "{}");"""
                           .format(date, place, temperature, rainfall, date, place, temperature, rainfall))
            
    def update_matches_teams_table(self, df_matches_teams):
        for i in range(len(df_matches_teams)):
                match_id = self.cur.execute("SELECT id FROM matches WHERE date = '{}' AND place = '{}';".format(
                    df_matches_teams.loc[i, 'date'], df_matches_teams.loc[i, 'place'])).fetchone()[0]
                team_id = self.cur.execute("SELECT id FROM teams WHERE city = '{}';".format(
                    df_matches_teams.loc[i, 'team'])).fetchone()[0]
                home = df_matches_teams.loc[i, 'team'] == df_matches_teams.loc[i, 'place']
                team_goal = df_matches_teams.loc[i, 'team_goal']
                points = 99

                self.cur.execute("""INSERT INTO matches_teams(match_id, team_id, home, team_goal, points) 
            SELECT "{}", "{}", "{}", "{}", "{}" 
            WHERE NOT EXISTS(SELECT * FROM matches_teams WHERE match_id = "{}" AND team_id = "{}" AND home = "{}" 
            AND team_goal = "{}" AND points = "{}");"""
                               .format(match_id, team_id, home, team_goal, points, match_id, team_id, home, team_goal, points))

                
    # find missing players
    def add_missing_players(self, df_matches_teams):
        count = 0
        players_not_in_list = {'name':[], 'team': []}
        for i in range(len(df_matches_teams)):
            for (player_name, goal_time, goal_type) in df_matches_teams.loc[i, 'players']:

                team_id = self.cur.execute("SELECT id FROM teams WHERE city = '{}';".format(
                            df_matches_teams.loc[i, 'team'])).fetchone()[0]
                #print(goal_type)
                if goal_type == "(csc)":
                    match_id = self.cur.execute("SELECT id FROM matches WHERE date = '{}' AND place = '{}';".format(
                            df_matches_teams.loc[i, 'date'], df_matches_teams.loc[i, 'place'])).fetchone()[0]
                    team_id = self.cur.execute("SELECT team_id FROM matches_teams WHERE team_id != '{}' AND match_id = '{}';".format(
                            team_id, match_id)).fetchone()[0]


                #print(player_name)
                try:
                    player_id = self.cur.execute(
                        """SELECT id FROM players WHERE name = "{}";""".format(player_name)).fetchone()[0]
                except:
                    if player_name not in players_not_in_list['name']:
                        players_not_in_list["name"].append(player_name)
                        players_not_in_list["team"].append(team_id)
                        count += 1

        print(f"Missing number of players: {count}")
        df_players_not_in_list = pd.DataFrame(players_not_in_list)

        # insert missing players
        for (name, team_id) in zip(df_players_not_in_list['name'], df_players_not_in_list['team']):
            position = None
            birthday = None
            nationality = None

            self.cur.execute(
                """INSERT INTO players(name, team_id, position, birthday, nationality) 
        SELECT "{}", "{}", "{}", "{}", "{}" 
        WHERE NOT EXISTS(SELECT * FROM players WHERE name = "{}" AND team_id = "{}" AND position = "{}" 
        AND birthday = "{}" AND nationality = "{}");""".format(name, team_id, position,
                                                               birthday, nationality,
                                                               name, team_id, position,
                                                               birthday, nationality))
            
    def update_goals_table(self, matches_teams):

        for i in range(len(df_matches_teams)):
            club = df_matches_teams.loc[i, 'team']
            for (player_name, goal_time, goal_type) in df_matches_teams.loc[i, 'players']:
                player_id = self.cur.execute(
                    """SELECT id FROM players WHERE name = "{}";""".format(player_name)).fetchone()[0]
                match_id = self.cur.execute(
                    """SELECT id FROM matches WHERE date = "{}" AND place = "{}";
                    """.format(df_matches_teams.loc[i, 'date'],
                           df_matches_teams.loc[i, 'place'])).fetchone()[0]
                self.cur.execute("""INSERT INTO goals(player_id, match_id, goal_time, goal_type) 
            SELECT ?, ?, ?, ?  WHERE NOT EXISTS(SELECT * FROM goals WHERE player_id = ? AND match_id = ? AND goal_time = ? 
        AND goal_type = ?);
            """,(player_id, match_id, goal_time, goal_type, player_id,
                                       match_id, goal_time, goal_type))
                

In [133]:
db = ChampionshipsDataBase()

<IPython.core.display.Javascript object>

In [134]:
db.get_tables_names()

['championships',
 'sqlite_sequence',
 'teams',
 'participation',
 'matches',
 'matches_teams',
 'players',
 'goals']

#### create championship table sql

In [135]:
db.update_championships_table()
db.get_df('championships')

,id,name,country,start_year,end_year
0,1,Ligue 1,France,2020,2021


#### create teams table in sql

In [136]:
db.update_teams_table(df_teams)
db.get_df('teams')

,id,name,city,coach_name,year_founded
0,1,Football Club des Girondins de Bordeaux,Bordeaux,Jean-Louis GASSET,1881
1,2,Dijon Football Côte-d'Or,Dijon,David LINARES,1998
2,3,LOSC Lille,Lille,Christophe GALTIER,1944
3,4,Association sportive de Monaco FC,Monaco,Niko KOVAC,1919
4,5,Football Club Lorient-Bretagne Sud,Lorient,Christophe PELISSIER,1926
5,6,Nîmes Olympique,Nîmes,Pascal PLANCQUE,1937
6,7,Olympique Gymnaste Club Nice Côte d'Azur,Nice,Adrian URSEA,1904
7,8,Montpellier-Herault Sport Club,Montpellier,Olivier DALL'OGLIO,1974
8,9,Paris-Saint-Germain Football Club,Paris-SG,Mauricio POCHETTINO,1970
9,10,Olympique de Marseille,Marseille,Jorge SAMPAOLI,1899


#### create participation table  

In [137]:
db.update_participation_table(df_teams)
db.get_df('participation').tail(3)

,id,championship_id,team_id
17,18,1,18
18,19,1,19
19,20,1,20


#### create players table 

In [138]:
db.update_players_table(df_players)
db.get_df('players')

,id,name,team_id,position,birthday,nationality
0,1,B. Costil,1,Gar.,1987-07-03,FRA
1,2,D. Lima Semedo,1,Gar.,2002-04-27,FRA
2,3,C. Michel,1,Gar.,2000-03-20,BEL
3,4,G. Poussin,1,Gar.,1999-01-13,FRA
4,5,D. Rouyard,1,Gar.,1999-08-17,FRA
...,...,...,...,...,...,...
642,643,I. Ganago,20,Att.,1999-02-16,CAM
643,644,C. Jean,20,Att.,1995-07-15,FRA
644,645,A. Kalimuendo,20,Att.,2002-01-20,FRA
645,646,F. Sotoca,20,Att.,1990-10-25,FRA


#### create matches table 

In [139]:
db.update_matches_table(df_matches)
db.get_df('matches')

,id,date,place,rainfall,temperature
0,1,2020-08-21 19:00:00,Bordeaux,0.0,18.0
1,2,2020-08-22 17:00:00,Dijon,0.0,18.0
2,3,2020-08-22 21:00:00,Lille,0.0,18.0
3,4,2020-08-23 13:00:00,Monaco,0.0,18.0
4,5,2020-08-23 15:00:00,Lorient,0.0,18.0
...,...,...,...,...,...
375,376,2021-05-23 21:00:00,Rennes,0.0,18.0
376,377,2021-05-23 21:00:00,Nantes,0.0,18.0
377,378,2021-05-23 21:00:00,Saint-Étienne,0.0,18.0
378,379,2021-05-23 21:00:00,Reims,0.0,18.0


#### create matches_teams table 

In [140]:
df_matches_teams.head()

,date,place,team,team_goal,players
0,2020-08-21 19:00:00,Bordeaux,Bordeaux,0,[]
1,2020-08-21 19:00:00,Bordeaux,Nantes,0,[]
2,2020-08-22 17:00:00,Dijon,Dijon,0,[]
3,2020-08-22 17:00:00,Dijon,Angers,1,"[(I. Traoré, 22’, )]"
4,2020-08-22 21:00:00,Lille,Rennes,1,"[(D. Da Silva, 74’, )]"


In [141]:
db.update_matches_teams_table(df_matches_teams)
db.get_df('matches_teams').tail()

,id,match_id,team_id,home,team_goal,points
755,756,373,20,True,0,99
756,757,372,19,True,0,99
757,758,371,17,True,1,99
758,759,379,1,False,2,99
759,760,380,5,False,1,99


#### create goals table 

In [142]:
db.add_missing_players(df_matches_teams)

Missing number of players: 12


In [143]:
db.update_goals_table(df_matches_teams)

In [144]:
db.get_df('players').shape

(659, 6)

In [145]:
goals = db.get_df('goals')

In [146]:
goals[goals['goal_type'] == "(csc)"]

,id,player_id,match_id,goal_time,goal_type
20,21,44,11,45’,(csc)
24,25,250,12,22’,(csc)
92,93,597,36,34’,(csc)
103,104,463,41,43’,(csc)
173,174,654,64,64’,(csc)
204,205,496,75,78’,(csc)
228,229,315,87,22’,(csc)
239,240,78,89,41’,(csc)
246,247,363,93,73’,(csc)
254,255,628,95,79’,(csc)


In [147]:
goals[goals['goal_type'] == "(s.p.)"]

,id,player_id,match_id,goal_time,goal_type
8,9,154,5,60’,(s.p.)
15,16,636,7,11’,(s.p.)
19,20,353,11,39’,(s.p.)
22,23,353,11,66’,(s.p.)
26,27,223,13,37’,(s.p.)
...,...,...,...,...,...
1020,1021,31,366,32’,(s.p.)
1024,1025,158,369,31’,(s.p.)
1035,1036,96,371,45’ +1,(s.p.)
1036,1037,327,375,90’ +13,(s.p.)
